In [1]:
import sys
import torch

# add the unet helpers
sys.path.append('../../')
from test_unet_helpers import *
from unet_models import unet11_MS
from utils import variable

from gbdxtools import CatalogImage

/home/ubuntu/anaconda3/envs/spatial_torch/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/anaconda3/envs/spatial_torch/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/anaconda3/envs/spatial_torch/lib/python3.5/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/spatial_torch/lib/python3.5/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/spatial_torch/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expe

In [2]:
## write out with gdal
import gdal, osr

def array2raster(newRasterfn,rasterOrigin,pixelWidth,pixelHeight,array):

    cols = array.shape[1]
    rows = array.shape[0]
    originX = rasterOrigin[0]
    originY = rasterOrigin[1]

    driver = gdal.GetDriverByName('GTiff')
    outRaster = driver.Create(newRasterfn, cols, rows, 1, gdal.GDT_Float64)
    outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    outband = outRaster.GetRasterBand(1)
    outband.WriteArray(array)
    outRasterSRS = osr.SpatialReference()
    outRasterSRS.ImportFromEPSG(4326)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()

# RGB-NIR
model_paths = ['./files_MS_bgrn1/bgrn1_ms_ep75_step1200_b32.pt']

dg_scene_id = '1030010057062200' #2016
# dg_scene_id = '104001000AA4B500' #2015
# img_2m = CatalogImage(dg_scene_id, pansharpened=True, acomp=True)
img_2m = CatalogImage(dg_scene_id, pansharpened=False, acomp=True)
chips = img_2m.window_cover((512,512), pad=False)
    
# 2-19-2019 using batch 32 RGB-NIR model
for model_path in model_paths:
    
    torch.cuda.empty_cache()
    
    #model = unet11(pretrained=False) # B-G-R
    model = unet11_MS(num_bands=4, pretrained=False) 
    model.eval()
        
    # load on CPU
    if os.path.exists(model_path):
        state_dict = torch.load(str(model_path), map_location='cpu')
        epoch = state_dict['epoch']
        step = state_dict['step']
        new_state_dict = OrderedDict()
        for k, v in state_dict['model'].items():
            name = k[7:] # remove 'module.' of dataparallel
            new_state_dict[name]=v

        model.load_state_dict(new_state_dict)
        print('Restored model, epoch {}, step {:,}'.format(epoch, step))

    
    # iterate over the image windows, getting the 4 bands
    for i,chip in enumerate(chips):
        
        rname = './rastertiles_MS_2016/denver_2016_t{0:0>6}_{1}.tif'.format(i+1, dg_scene_id)
        if os.path.exists(rname):
            continue
        
        print('on {} of {}'.format(i+1, 27051))
        img_arr = chip[[1,2,4,6], :, :].compute()/10000 #B-G-R-N1
        img_arr = torch.from_numpy(img_arr)
        #img_arr = img_transform(torch.from_numpy(img_arr))
        
        #input_img = torch.unsqueeze(variable(img_arr, volatile=True), dim=0)
        input_img = torch.unsqueeze(variable(img_arr, volatile=True), dim=0).cpu()
        del img_arr

        # model inference
        big_mask = model(input_img)
        #print(big_mask.cpu().detach().numpy().squeeze().max())

        # file creation
        #raster_id = os.path.basename(model_path).split('.')[0]
        #rname = '../tiles_64x64/denver_2016_t{0:0>6}_{1}.tif'.format(i+1, dg_scene_id)
        aff = chip.affine
        raster_origin = (aff.c, aff.f)
        pixel_height = aff.e
        pixel_width = aff.a
        array2raster(rname, raster_origin, pixel_width, pixel_height, big_mask.cpu().detach().numpy().squeeze())

        del input_img
        del big_mask
        

        torch.cuda.empty_cache()

Exception: Could not find a catalog entry for the given id: 1030010057062200

In [4]:
from gbdxtools import Interface
gbdx = Interface() 
gbdx.ordering.order('1030010057062200')

'0e6abc70-0033-44af-a8a7-70c56b1f3f96'

In [13]:
CatalogImage('1030010057062200')

Exception: Could not find a catalog entry for the given id: 1030010057062200